###  Create a response table using qgrid to pick the data and pivot table to format

In [30]:
import numpy as np
import pandas as pd
import os
import gcsfs
import xarray as xr

In [31]:
import pandas as pd

import qgrid
dfcat = pd.read_csv('https://cmip6.storage.googleapis.com/cmip6-zarr-consolidated-stores-noQC.csv')

In [32]:
variables = ['hfls','snw','mrro','mrso','mrsos']
exps = ['historical', 'ssp126', 'ssp245', 'ssp370', 'ssp585']

# define a simple search on keywords
def search_df(df, verbose= False, **search):
    "search by keywords - if list, then match exactly, otherwise match as substring"
    keys = ['activity_id','institution_id','source_id','experiment_id','member_id', 'table_id', 'variable_id', 'grid_label']
    d = df
    for skey in search.keys():
        
        if isinstance(search[skey], str):  # match a string as a substring
            d = d[d[skey].str.contains(search[skey])]
        else:
            dk = []
            for key in search[skey]:       # match a list of strings exactly
                dk += [d[d[skey]==key]]
            d = pd.concat(dk)
            keys.remove(skey)
    if verbose:
        for key in keys:
            print(key,' = ',list(d[key].unique()))      
    return d

dfs = search_df(dfcat,variable_id=variables,experiment_id=exps,table_id='mon')

In [40]:
widget=qgrid.show_grid(dfcat)
widget 

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [41]:
# save results
df = widget.get_changed_df()     # use this to save filtered dataframe
# df = widget.get_selected_df()  # use this instead to save selected rows
df.source_id.unique(), len(df)

(array(['BCC-CSM2-MR', 'BCC-ESM1', 'CAMS-CSM1-0', 'FGOALS-g3', 'CanESM5',
        'IITM-ESM', 'CNRM-CM6-1', 'CNRM-ESM2-1', 'ACCESS-CM2',
        'ACCESS-ESM1-5', 'E3SM-1-0', 'E3SM-1-1-ECA', 'E3SM-1-1',
        'EC-Earth3-Veg', 'EC-Earth3', 'FIO-ESM-2-0', 'IPSL-CM6A-LR',
        'MIROC-ES2L', 'MIROC6', 'HadGEM3-GC31-LL', 'HadGEM3-GC31-MM',
        'UKESM1-0-LL', 'MPI-ESM1-2-HR', 'MRI-ESM2-0', 'GISS-E2-1-G-CC',
        'GISS-E2-1-G', 'GISS-E2-1-H', 'GISS-E2-2-G', 'CESM2-FV2',
        'CESM2-WACCM-FV2', 'CESM2-WACCM', 'CESM2', 'NorESM1-F', 'GFDL-CM4',
        'GFDL-ESM4', 'NESM3', 'SAM0-UNICON', 'CIESM', 'MCM-UA-1-0'],
       dtype=object), 48)

In [42]:
print(sorted(df.source_id.unique()))

['ACCESS-CM2', 'ACCESS-ESM1-5', 'BCC-CSM2-MR', 'BCC-ESM1', 'CAMS-CSM1-0', 'CESM2', 'CESM2-FV2', 'CESM2-WACCM', 'CESM2-WACCM-FV2', 'CIESM', 'CNRM-CM6-1', 'CNRM-ESM2-1', 'CanESM5', 'E3SM-1-0', 'E3SM-1-1', 'E3SM-1-1-ECA', 'EC-Earth3', 'EC-Earth3-Veg', 'FGOALS-g3', 'FIO-ESM-2-0', 'GFDL-CM4', 'GFDL-ESM4', 'GISS-E2-1-G', 'GISS-E2-1-G-CC', 'GISS-E2-1-H', 'GISS-E2-2-G', 'HadGEM3-GC31-LL', 'HadGEM3-GC31-MM', 'IITM-ESM', 'IPSL-CM6A-LR', 'MCM-UA-1-0', 'MIROC-ES2L', 'MIROC6', 'MPI-ESM1-2-HR', 'MRI-ESM2-0', 'NESM3', 'NorESM1-F', 'SAM0-UNICON', 'UKESM1-0-LL']


In [43]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

dm = df[['experiment_id','source_id','variable_id','member_id']].groupby([
         'experiment_id','source_id','variable_id']).nunique()[['member_id']]

table = pd.DataFrame.pivot_table(dm,
                                 values='member_id',
                                 index=['source_id','variable_id'],
                                 columns=['experiment_id'],
                                 aggfunc=np.sum,
                                 fill_value=0)
print('activity_id, table_id = ',df.activity_id.unique(),df.table_id.unique())
display(table)

activity_id, table_id =  ['CMIP'] ['Amon']


,experiment_id,piControl
source_id,variable_id,
ACCESS-CM2,tas,1
ACCESS-ESM1-5,tas,1
BCC-CSM2-MR,tas,1
BCC-ESM1,tas,1
CAMS-CSM1-0,tas,1
CESM2,tas,1
CESM2-FV2,tas,1
CESM2-WACCM,tas,1
CESM2-WACCM-FV2,tas,1


In [44]:
dm = df[['table_id','experiment_id','source_id','variable_id','member_id']].groupby([
         'table_id','experiment_id','source_id','variable_id']).nunique()[['member_id']]

table = pd.DataFrame.pivot_table(dm,
                                 values='member_id',
                                 index=['experiment_id','source_id'],
                                 columns=['variable_id'],
                                 aggfunc=np.sum,
                                 fill_value=0)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

print('current CMIP6 datasets, number of ensemble members available')
display(table)

current CMIP6 datasets, number of ensemble members available


variable_id                    tas
experiment_id source_id           
piControl     ACCESS-CM2         1
              ACCESS-ESM1-5      1
              BCC-CSM2-MR        1
              BCC-ESM1           1
              CAMS-CSM1-0        1
              CESM2              1
              CESM2-FV2          1
              CESM2-WACCM        1
              CESM2-WACCM-FV2    1
              CIESM              1
              CNRM-CM6-1         1
              CNRM-ESM2-1        1
              CanESM5            2
              E3SM-1-0           1
              E3SM-1-1           1
              E3SM-1-1-ECA       1
              EC-Earth3          1
              EC-Earth3-Veg      1
              FGOALS-g3          1
              FIO-ESM-2-0        1
              GFDL-CM4           1
              GFDL-ESM4          1
              GISS-E2-1-G        7
              GISS-E2-1-G-CC     1
              GISS-E2-1-H        1
              GISS-E2-2-G        1
              HadGEM3-GC31-LL    1
              HadGEM3-GC31-MM    1
              IITM-ESM           1
              IPSL-CM6A-LR       2
              MCM-UA-1-0         1
              MIROC-ES2L         1
              MIROC6             1
              MPI-ESM1-2-HR      1
              MRI-ESM2-0         2
              NESM3              1
              NorESM1-F          1
              SAM0-UNICON        1
              UKESM1-0-LL        1

current CMIP6 datasets


variable_id,hfls,mrro,mrso,mrsos,snw
experiment_id,,,,,
historical,297,279,276,276,150
ssp126,110,110,105,85,83
ssp245,137,138,132,49,127
ssp370,142,138,135,43,128
ssp585,114,111,106,46,99


current CMIP6 datasets


variable_id,hfls,mrro,mrso,mrsos,snw
experiment_id,,,,,
historical,297,279,276,171,150
ssp126,109,108,103,36,34
ssp245,131,131,125,49,39
ssp370,137,131,105,43,41
ssp585,111,108,84,46,35


In [31]:
dm = df[['experiment_id','source_id','variable_id','member_id']].groupby([
         'experiment_id','source_id','variable_id']).nunique()[['member_id']]

table = pd.DataFrame.pivot_table(dm,
                                 values='member_id',
                                 index=['experiment_id','source_id'],
                                 columns=['variable_id'],
                                 aggfunc=np.sum,
                                 fill_value=0)

print('current CMIP6 datasets')
display(table)

current CMIP6 datasets


variable_id                    so  thetao  uo  vo
experiment_id source_id                          
piControl     ACCESS-CM2        0       0   0   1
              AWI-CM-1-1-MR     1       1   1   1
              BCC-CSM2-MR       1       1   1   1
              BCC-ESM1          1       1   1   1
              CAMS-CSM1-0       1       1   1   1
              CESM2             1       1   1   1
              CESM2-WACCM       1       1   1   1
              CNRM-CM6-1        1       1   1   1
              CNRM-CM6-1-HR     0       1   0   1
              CNRM-ESM2-1       1       1   1   1
              CanESM5           1       1   1   1
              E3SM-1-0          1       1   1   1
              E3SM-1-1          1       1   1   1
              EC-Earth3         1       1   1   1
              EC-Earth3-LR      1       1   1   1
              EC-Earth3-Veg     1       1   1   1
              FIO-ESM-2-0       0       1   0   0
              GFDL-CM4          1       1   1   1
              GFDL-ESM4         1       1   0   0
              GISS-E2-1-G       1       2   1   1
              GISS-E2-1-G-CC    1       1   1   1
              GISS-E2-1-H       1       1   1   1
              GISS-E2-2-G       0       0   0   1
              HadGEM3-GC31-LL   1       1   1   1
              IPSL-CM6A-LR      1       1   1   1
              MCM-UA-1-0        1       1   1   1
              MIROC-ES2L        1       1   1   1
              MIROC6            1       1   1   1
              NESM3             1       1   1   1
              NorESM2-LM        0       1   0   0
              SAM0-UNICON       1       1   1   1
              UKESM1-0-LL       1       1   0   0

In [142]:
df.variable_id.unique()

array(['huss', 'tas', 'uas', 'vas', 'hus', 'ps', 'ta', 'ua', 'va'],
      dtype=object)

In [179]:
dm = df[['experiment_id','source_id','variable_id','member_id']].groupby([
         'experiment_id','source_id','variable_id']).nunique()[['member_id']]

table = pd.DataFrame.pivot_table(dm,
                                 values='member_id',
                                 index=['experiment_id','source_id'],
                                 columns=['variable_id'],
                                 aggfunc=np.sum,
                                 fill_value=0)

print('current CMIP6 datasets')
display(table)

current CMIP6 datasets


variable_id                        fgco2  intpp  sos  tos
experiment_id  source_id                                 
dcppA-hindcast BCC-CSM2-MR             0      0    1    1
               CESM1-1-CAM5-CMIP5     40      0   40   40
               CanESM5                20      0   20   20
               MIROC6                  0      0   10   10
               MPI-ESM1-2-HR           5      5    0    0
               NorCPM1                20      0   20   20
esm-piControl  BCC-CSM2-MR             1      0    0    1
               CESM2                   1      1    0    1
               CNRM-ESM2-1             1      1    0    1
               CanESM5                 1      0    0    1
               GFDL-ESM4               1      1    0    1
               MRI-ESM2-0              0      1    0    0
               NorESM2-LM              1      0    0    1
piControl      AWI-CM-1-1-MR           0      0    0    1
               BCC-CSM2-MR             0      0    0    1
               BCC-ESM1                1      0    0    1
               CAMS-CSM1-0             0      0    0    1
               CESM2                   1      1    0    1
               CESM2-FV2               1      1    0    1
               CESM2-WACCM             1      1    0    1
               CESM2-WACCM-FV2         1      1    0    1
               CNRM-CM6-1              0      0    0    1
               CNRM-CM6-1-HR           0      0    0    1
               CNRM-ESM2-1             1      1    0    1
               CanESM5                 2      0    0    2
               E3SM-1-0                0      0    0    1
               E3SM-1-1                0      0    0    1
               EC-Earth3               0      0    0    1
               EC-Earth3-LR            0      0    0    1
               EC-Earth3-Veg           0      0    0    1
               FIO-ESM-2-0             0      0    0    1
               GFDL-CM4                0      0    0    1
               GFDL-ESM4               1      1    0    1
               GISS-E2-1-G             2      2    0    6
               GISS-E2-1-G-CC          1      1    0    1
               GISS-E2-1-H             0      0    0    1
               GISS-E2-2-G             0      0    0    1
               HadGEM3-GC31-LL         0      0    0    1
               IPSL-CM6A-LR            2      2    0    2
               MCM-UA-1-0              0      0    0    1
               MIROC-ES2L              1      1    0    1
               MIROC6                  0      0    0    1
               MRI-ESM2-0              0      1    0    1
               NESM3                   0      0    0    1
               NorESM1-F               1      0    0    1
               SAM0-UNICON             0      0    0    1
               UKESM1-0-LL             0      1    0    1

In [34]:
table[(table.chl>0)&(table.dissic>0)]

variable_id                            chl  chldiazos  chlmiscos  chlos  dissic  dissoc  dissocos
table_id experiment_id source_id                                                                 
Omon     historical    CESM2            11          1          0      1      11       1         1
                       CESM2-WACCM       1          1          0      1       3       1         1
                       CNRM-ESM2-1       1          0          0      0       5       0         0
                       CanESM5           1          0          0      1      35       0         0
                       GFDL-CM4          1          0          0      0       1       1         0
                       GISS-E2-1-G       1          0          0      0       2       0         0
                       GISS-E2-1-G-CC    1          1          1      1       1       1         1
                       IPSL-CM6A-LR      1          0          1      1      32       1         0
                       MIROC-ES2L        1          0          0      0       3       0         0
                       UKESM1-0-LL       1          0          0      0       3       0         0
         ssp126        CESM2             1          1          0      1       1       1         1
                       CESM2-WACCM       1          1          0      1       1       1         1
                       CNRM-ESM2-1       1          0          0      0       1       0         0
                       CanESM5           1          0          0      1       2       0         0
                       IPSL-CM6A-LR      1          0          1      1       1       1         1
                       MIROC-ES2L        1          0          0      0       1       0         0
                       MPI-ESM1-2-HR     1          1          0      1       1       1         1
                       UKESM1-0-LL       1          0          0      0       1       0         0
         ssp245        CESM2             1          1          0      1       1       1         1
                       CESM2-WACCM       1          1          0      1       1       1         1
                       CNRM-ESM2-1       1          0          0      0       1       0         0
                       CanESM5           1          0          0      1       2       0         0
                       GFDL-CM4          1          0          0      0       1       1         0
                       IPSL-CM6A-LR      1          0          1      1       1       1         1
                       MIROC-ES2L        1          0          0      0       1       0         0
                       UKESM1-0-LL       1          0          0      0       1       0         0
         ssp585        CESM2             2          0          0      0       2       0         0
                       CESM2-WACCM       1          0          0      0       1       0         0
                       CanESM5           1          0          0      0       1       0         0
                       GFDL-CM4          1          0          0      0       1       0         0
                       IPSL-CM6A-LR      1          0          0      0       1       0         0
                       MIROC-ES2L        1          0          0      0       1       0         0
                       MPI-ESM1-2-HR     1          0          0      0       1       0         0
                       UKESM1-0-LL       1          0          0      0       1       0         0

In [40]:
#xr.open_zarr('/h63/naomi/zarr-minimal/AerChemMIP/MOHC/UKESM1-0-LL/piClim-control/r1i1p1f2/AERmon/mmroa/gn')

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 144, lev: 85, lon: 192, time: 540)
Coordinates:
    b_bnds     (lev, bnds) float64 dask.array<chunksize=(85, 2), meta=np.ndarray>
  * lat        (lat) float64 -89.38 -88.12 -86.88 -85.62 ... 86.88 88.12 89.38
    lat_bnds   (lat, bnds) float64 dask.array<chunksize=(144, 2), meta=np.ndarray>
  * lev        (lev) float64 20.0 53.33 100.0 ... 7.378e+04 7.91e+04 8.5e+04
    lev_bnds   (lev, bnds) float64 dask.array<chunksize=(85, 2), meta=np.ndarray>
  * lon        (lon) float64 0.9375 2.812 4.688 6.562 ... 355.3 357.2 359.1
    lon_bnds   (lon, bnds) float64 dask.array<chunksize=(192, 2), meta=np.ndarray>
  * time       (time) object 1850-01-16 00:00:00 ... 1894-12-16 00:00:00
    time_bnds  (time, bnds) object dask.array<chunksize=(540, 2), meta=np.ndarray>
Dimensions without coordinates: bnds
Data variables:
    b          (lev) float64 dask.array<chunksize=(85,), meta=np.ndarray>
    mmroa      (time, lev, lat, lon) float32 dask.array<chun